# TITLE

(some sort of image)

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Abstract

Text

##  Table of Contents

* [Part 1: Creating the Data Set](#Part-1:-Creating-the-Data-Set)
    * [Part 1: Creating the Data Set](#Part-1:-Creating-the-Data-Set)
* [Part 1: Creating the Data Set](#Part-1:-Creating-the-Data-Set)
    * [Part 1: Creating the Data Set](#Part-1:-Creating-the-Data-Set)
* [Part 1: Creating the Data Set](#Part-1:-Creating-the-Data-Set)
    * [Part 1: Creating the Data Set](#Part-1:-Creating-the-Data-Set)

## Part 1: Creating the Data Set

### Concatenating the Data Files

The Q data we received consisted of about 2300 JSON files, each one representing a unique course's historical Q scores for each available semester. Out first task was to combine these individual JSON files into a single JSON Lines file; the JSON files were stored locally outside of the repository directory, and we used the `shututil` and `glob`, and libraries to concatenate these into `qall.jsonl`.

In [2]:
import shutil
import glob

# Opens the JSONL file as output.
with open("qall.jsonl", "wb") as outfile:
    
    # Iterates over the JSON files.
    for filename in glob.glob("../Q/*.json"):
        
        # Opens the current JSON file as input.
        with open(filename, "rb") as readfile:
            
            # Copies the JSON file to the JSONL file.
            shutil.copyfileobj(readfile, outfile)
            
            # Adds a new line as a delimiter.
            outfile.write("\n")

### Loading the Data

Each JSON file was structured using multiple nested dictionaries, with the following format:

* `scores`: a dictionary of mostly numerical data, separated by semester
    * `*semester*`: a dictionary of course and instructor scores
        * `course_score`: a dictionary of course info, ratings, and rating breakdowns
            * `overall`: a float of mean overall rating
            * `workload`: a float of mean workload rating
            * `difficulty`: a float of mean difficulty rating
            * `recommendation`: a float of mean recommendation rating
            * `enrollment`: an integer of class enrollment
            * `reponse rate`: a float of the proportion of students who reviewed the course
            * `QCourseOverall`: 
                * `number`: the string concatenation of department and course number, i.e. "COMPSCI 109"
                * `course_id`: an integer identifier for my.harvard.edu
                * `cat_num`: an integer indentifier for the Harvard Course Catalog
                * `1s`: an integer count of reviews with rating 1 for this category
                * `2s`: analagous, with 2s
                * `3s`: analagous, with 3s
                * `4s`: analagous, with 4s
                * `5s`: analagous, with 5s
            * `QDifficulty`: analagous to `QCourseOverall`, with difficulty ratings.
            * `QWorkload`: analagous to `QCourseOverall`, with workload ratings.
        * `instructor_scores`: a list of length 1 that contains a dictionary of instructor info and ratings
            * `number`: identical to that within `QCourseOverall`
            * `cat_num`: identical to that within `QCourseOverall`
            * `course_id`: identical to that within `QCourseOverall`
            * `year`: an int of the first of the two years represented in the given school year
            * `term`: an int with value 1 if the course took place in the Fall, 2 if Spring
            * `id`: a string indentifier of the professor
            * `first`: a string of the first name of the professor
            * `last`: a string of the last name
            * `InstructorOverall`: a float of mean overall instructor rating
            * `EffectiveLecturesorPresentations`: a float of mean lecture rating
            * `AccessibleOutsideClass`: a float of mean accessibility rating
            * `GeneratesEnthusiasm`: a float of mean enthusiasm rating
            * `FacilitatesDiscussionEncouragesParticipation`: a float of mean facilitation rating
            * `GivesUsefulFeedback`: a float of mean feedback rating
            * `ReturnsAssignmentsinTimelyFashion`: a float of mean timeliness rating
* `comments`: a dictionary of textual reviews, separated by semester into dictionaries
    * `*semester*`: a dictionary of comments for the given semester
        * `comments`: a list of strings that represent comments
* `mostRecentQ`: identical to the most recent semester data in `scores`
* `success`: indicates if the JSON was successfully retrieved

Any mean rating has range 1.0-5.0, and response rate has a range 0.0-1.0. The entries labeled `*semester*` can occur any number of times and have as a key the semester in which it took place (i.e. "Fall '12").

Some caveats of the data include...

In [ ]:
import json

# Creates the dataframe with columns specified.
bigdf = pd.DataFrame(columns=["C_Department","C_Number", "Course", "C_CatNum","C_ID","C_Semester","C_Year","C_Term",
                              "C_Overall","C_Workload","C_Difficulty","C_Recommendation","C_Enrollment","C_ResponseRate",
                              "I_First","I_Last","I_ID",
                              "I_Overall","I_EffectiveLectures","I_Accessible","I_GeneratesEnthusiasm",
                              "I_EncouragesParticipation","I_UsefulFeedback","I_ReturnsAssignmentsTimely",
                              "QOverall_1","QOverall_2","QOverall_3","QOverall_4","QOverall_5",
                              "QDifficulty_1","QDifficulty_2","QDifficulty_3","QDifficulty_4","QDifficulty_5",
                              "QWorkload_1","QWorkload_2","QWorkload_3","QWorkload_4","QWorkload_5",
                              "Comments"])

# Opens the JSONL file as the input.
with open("qall.jsonl") as data_file:

    # Iterates over each line.
    for line in data_file:

        # Loads the JSON.
        alldata = json.loads(line)

        # Iterates over each semester for the course.
        for semester in alldata["scores"]:

            # Retrieves the semester data.
            semdata = alldata["scores"][semester]

            # Retrieves the scores or comments if they exist, else None.
            cscores = semdata["course_score"] if "course_score" in semdata else None
            iscores = semdata["instructor_scores"][0] if "instructor_scores" in semdata else None
            comments = alldata["comments"][semester]["comments"] if semester in alldata["comments"] else None

            # Creates a dictionary that will be appended as a row to the dataframe.
            semdatadict = {"C_Semester": semester, "Comments": comments}

            # If the course scores exist...
            if cscores != None:
                
                # Retrieves the course name and separates it into department and number.
                course = str(cscores["QCourseOverall"]["number"]).split()
                if course[0] == "None":
                    course = [None, None]
                    fullCourseName = "NODEPT-NOCOURSENUMBER"
                else:
                    fullCourseName = course[0] + "-" + course[1]
                    
                # Updates the dictionary with the data.
                semdatadict.update({"C_Department": course[0],
                                    "C_Number": course[1],
                                    "Course": fullCourseName,
                                    "C_CatNum": cscores["QCourseOverall"]["cat_num"],
                                    "C_ID": cscores["QCourseOverall"]["course_id"],
                                    "C_Overall": cscores["overall"],
                                    "C_Workload": cscores["workload"],
                                    "C_Difficulty": cscores["difficulty"],
                                    "C_Recommendation": cscores["recommendation"],
                                    "C_Enrollment": cscores["enrollment"],
                                    "C_ResponseRate": cscores["response rate"],
                                    "QOverall_1": cscores["QCourseOverall"]["1s"],
                                    "QOverall_2": cscores["QCourseOverall"]["2s"],
                                    "QOverall_3": cscores["QCourseOverall"]["3s"],
                                    "QOverall_4": cscores["QCourseOverall"]["4s"],
                                    "QOverall_5": cscores["QCourseOverall"]["5s"],
                                    "QDifficulty_1": cscores["QDifficulty"]["1s"],
                                    "QDifficulty_2": cscores["QDifficulty"]["2s"],
                                    "QDifficulty_3": cscores["QDifficulty"]["3s"],
                                    "QDifficulty_4": cscores["QDifficulty"]["4s"],
                                    "QDifficulty_5": cscores["QDifficulty"]["5s"],
                                    "QWorkload_1": cscores["QWorkload"]["1s"],
                                    "QWorkload_2": cscores["QWorkload"]["2s"],
                                    "QWorkload_3": cscores["QWorkload"]["3s"],
                                    "QWorkload_4": cscores["QWorkload"]["4s"],
                                    "QWorkload_5": cscores["QWorkload"]["5s"],})

            # If the instructor scores exist...
            if iscores != None:

                # Retrieves the course name and separates it into department and number.
                course = str(iscores["number"]).split()
                if course[0] == "None":
                    course = [None, None]
                    fullCourseName = "NODEPT-NOCOURSENUMBER"
                else:
                    fullCourseName = course[0] + "-" + course[1]

                # Updates the dictionary with the data.
                semdatadict.update({"C_Department": course[0],
                                    "C_Number": course[1],
                                    "Course": fullCourseName,
                                    "C_CatNum": iscores["cat_num"],
                                    "C_ID": iscores["course_id"],
                                    "C_Year": iscores["year"],
                                    "C_Term": iscores["term"],
                                    "I_First": iscores["first"].strip(),
                                    "I_Last": iscores["last"].strip(),
                                    "I_ID": iscores["id"],
                                    "I_Overall": iscores["InstructorOverall"],
                                    "I_EffectiveLectures": iscores["EffectiveLecturesorPresentations"],
                                    "I_Accessible": iscores["AccessibleOutsideClass"],
                                    "I_GeneratesEnthusiasm": iscores["GeneratesEnthusiasm"],
                                    "I_EncouragesParticipation": iscores["FacilitatesDiscussionEncouragesParticipation"],
                                    "I_UsefulFeedback": iscores["GivesUsefulFeedback"],
                                    "I_ReturnsAssignmentsTimely": iscores["ReturnsAssignmentsinTimelyFashion"]})

            # Adds the dictionary as a row to the dataframe.
            bigdf = bigdf.append(semdatadict, ignore_index=True)

In [ ]:
bigdf.head(5)

In [ ]:
for index, row in bigdf.iterrows():

    year = int(row["C_Semester"][-2:])

    if row["C_Semester"][0:4] == "Fall":
        row["C_Term"] = 1
        row["C_Year"] = 2000 + year
    elif row["C_Semester"][0:6] == "Spring":
        row["C_Term"] = 2
        row["C_Year"] = 1999 + year
    else:
        continue

In [ ]:
print bigdf[bigdf["C_Department"]!=bigdf["C_Department"]].shape[0]
print bigdf[bigdf["C_Department"]==None].shape[0]

In [ ]:
bigdf = bigdf[bigdf["C_Department"]==bigdf["C_Department"]]

In [ ]:
bigdf = bigdf.replace({"C_CatNum": {"": None}})

In [ ]:
coltypes = {int: ["C_CatNum","C_ID","C_Year","C_Term","C_Enrollment",
                  "QOverall_1","QOverall_2","QOverall_3","QOverall_4","QOverall_5",
                  "QDifficulty_1","QDifficulty_2","QDifficulty_3","QDifficulty_4","QDifficulty_5",
                  "QWorkload_1","QWorkload_2","QWorkload_3","QWorkload_4","QWorkload_5"],
            float: ["C_Overall","C_Workload","C_Difficulty","C_Recommendation","C_ResponseRate",
                    "I_Overall","I_EffectiveLectures","I_Accessible","I_GeneratesEnthusiasm",
                    "I_EncouragesParticipation","I_UsefulFeedback","I_ReturnsAssignmentsTimely",
                    "QOverall_1","QOverall_2","QOverall_3","QOverall_4","QOverall_5"],
            str: ["C_Number","C_Department","C_Semester","I_First","I_Last","I_ID"]}

In [ ]:
from unidecode import unidecode

for index, row in bigdf.iterrows():
    for strcol in coltypes[str]:
        try:
            str(row[strcol])
        except UnicodeEncodeError:
            bigdf.loc[index, row[row.values==row[strcol]].index[0]] = unidecode(row[strcol])

In [ ]:
for intcol in coltypes[int]:
    bigdf = bigdf.replace({intcol: {None: -1}})

for floatcol in coltypes[float]:
    bigdf = bigdf.replace({floatcol: {None: np.nan}})

In [ ]:
for ctp in coltypes:
    for col in coltypes[ctp]:
        bigdf[col] = bigdf[col].astype(ctp)

In [ ]:
# for index, comments in bigdf["Comments"].iteritems():
#     if comments != None:
#         bigdf["Comments"].loc[index] = [unidecode(c) for c in comments]

In [ ]:
bigdf["Sem_Average"] = pd.Series(index=bigdf.index)

In [ ]:
for name, group in bigdf.groupby(["C_Year", "C_Term"]):
    bigdf.loc[(bigdf["C_Year"]==name[0])&(bigdf["C_Term"]==name[1]), "Sem_Average"] = np.nanmean(group["C_Overall"].values)

In [ ]:
bigdf["Positive"] = pd.Series(bigdf["C_Overall"]>=bigdf["Sem_Average"], index=bigdf.index)

In [ ]:
backupdf = bigdf.copy()
bigdf = backupdf.copy()

In [ ]:
bigdf.head(5)

In [ ]:
bigdf.to_csv("bigdf.csv", index=False)